In [ ]:
## DriveEasy exploration notebook. suitable for data for 10 min, 200Hz.
## BIN_SIZE=50 corresponding to minimum event_distance = 0.25 sec. BIN_SIZE=25 fit for M80 better
## Event threshold 1 pm
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import scipy.signal as signal

FIBER_DISTANCE = 2.5
SAMPLING_RATE = 200
THRESHOLD = 0.001 # picometer
BIN_SIZE = 25


class Event:
    def __init__(self):
        self.timestamp = 0
        self.start = 0
        self.end = 1
        self.speed = 0
        self.index = 1
        self.ch = 1
        self.location = 1
        self.data = {}
        self.sensor = []
        self.sensor_active = []

    def calculate_speed(self, event_vis_list, sn=25, offset=0):
        speed_temp = []
        speed = 0
        self.active_sensor = []
#         self.speed_alg3([1,2])
        if (self.speed > 130) or (self.speed < 50):
            for i in self.sensor:
                if np.max(event_vis_list[int(self.start/BIN_SIZE)-1:int(self.end/BIN_SIZE)+1, i]) > 0:
                    self.active_sensor.append(i)
                speed = signal.correlate(np.asarray(self.data['leading'].iloc[:, i])-self.data['leading'].iloc[0,i],
                                         np.asarray(self.data['trailing'].iloc[:, i-offset])-self.data['trailing'].iloc[0,i-offset])
                speed_temp.append(speed.argmax())
            if len(speed_temp) < 1:
                print('Speed detection error_1!')
                self.speed = 0
            else:
                self.speed = -1*FIBER_DISTANCE/(np.median(speed_temp)-int(len(self.data['leading'])))*SAMPLING_RATE*3.6
                if np.abs(self.speed)>180:
                    print('Speed detection error_2!')
                    self.speed_alg2(self.sensor)
    def speed_alg2(self, active_sensor):
        leading = np.sum(self.data['leading'].iloc[:,active_sensor], axis=1)
        leading = np.abs(leading-leading[0])
        trailing = np.sum(self.data['trailing'].iloc[:,active_sensor], axis=1)
        trailing = np.abs(trailing-trailing[0])
        pk1 = signal.find_peaks(leading, height=0.005)

        pk2 = signal.find_peaks(trailing, height=0.005)
        if np.min([len(pk1[0]),len(pk2[0])]) == 0:
            print('Speed detection error_3!')
            self.speed = 0
        else:
            self.speed = FIBER_DISTANCE/(pk2[0][0]-pk1[0][0])*SAMPLING_RATE*3.6
        if np.abs(self.speed)>200:
                print('Speed detection error_4!')
                self.speed = 0
    def speed_alg3(self, active_sensor):
        leading = np.abs(self.data['leading'].iloc[:,active_sensor]-self.data['leading'].iloc[0,active_sensor])
        pk1 = signal.find_peaks(leading, height=0.002)
        trailing = np.abs(self.data['trailing'].iloc[:,active_sensor]-self.data['trailing'].iloc[0,active_sensor])
        pk2 = signal.find_peaks(trailing, height=0.002)
        if np.min([len(pk1[0]),len(pk2[0])]) == 0:
            print('Speed detection error_3!')
            self.speed = 0
        else:
            self.speed = FIBER_DISTANCE/(pk2[0][0]-pk1[0][0])*SAMPLING_RATE*3.6

    def calculate_location(self):
        pass

    def calculate_index(self):
        pass


def define_baseline_alg2(data_seg, moving_ave, threshold):
    return np.asarray(data_seg[:len(moving_ave)])-np.asarray(moving_ave) > threshold


def define_baseline_alg1(data_seg, threshold):
    if np.max(np.abs(data_seg - np.median(data_seg))) > threshold:
        return 1
    else:
        return 0


# define the minimum distance in time between two vehicles: BIN_SIZE
def event_detection(data_trace, threshold=0.001, seg_length=BIN_SIZE):
#     print('Process start')
#     start = time.process_time_ns()
    event_flag = []
    for i in range(int(len(data_trace)/seg_length)):
        event_flag.append(define_baseline_alg1(data_trace[i*seg_length:(i+1)*seg_length], threshold=threshold))
        # event_flag = define_baseline_alg2(data_trace, moving_ave, threshold=0.001)
#     print('Process end. Takes s: ')

#     print((time.process_time_ns()-start)/1e9)
    return event_flag


def moving_average(data_set, periods=3):
    weights = np.ones(periods) / periods
    return np.convolve(data_set, weights, mode='valid')


def de_trend(data, n_ave=200):
    trend = moving_average(data, periods=n_ave)
    return data[:-(n_ave - 1)] - trend

In [ ]:
filename = r'C:\Users\qchen\PARC\Fibridge-PARC - Drive Easy\AustraliaDeploy\M80\cleaned_data\Video2_20201118_0923AM\wavelength\wav_video2-9_18AM-9_23AM_ch_3.pickle'
filename_2 = r'C:\Users\qchen\PARC\Fibridge-PARC - Drive Easy\AustraliaDeploy\M80\cleaned_data\Video2_20201118_0923AM\wavelength\wav_video2-9_18AM-9_23AM_ch_4.pickle'

# ## pickle
#for pickle files
df = pd.read_pickle(filename)
df2 = pd.read_pickle(filename_2)
df['timestamp']=df.index
df2['timestamp']=df2.index
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df2 = df2[cols]
df = df.reset_index(drop=True)
df2 = df2.reset_index(drop=True)





## # npz
# data1 = np.load(filename, allow_pickle=True)
# data2 = np.load(filename_2, allow_pickle=True)

# df = pd.DataFrame(data1['wavelength'])
# df['sum'] = df.sum(axis=1)
# df_time = pd.DataFrame(data1['timestamp'], columns=['timestamp'])
# # df_time = df_time.iloc[::-1].reset_index(drop=True)  # fixed the reversed timestamp
# df = df.join(df_time)

# cols = df.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# df = df[cols]

# df2 = pd.DataFrame(data2['wavelength'])
# df2['sum'] = df2.sum(axis=1)
# df2_time = pd.DataFrame(data2['timestamp'], columns=['timestamp'])
# # df2_time = df2_time.iloc[::-1].reset_index(drop=True)  # fixed the reversed timestamp
# df2 = df2.join(df2_time, rsuffix='timestamp')
# df2 = df2[cols]


# # # csv
# file1 = r'C:\Users\qchen\PARC\Fibridge-PARC - Drive Easy\AustraliaDeploy\Francis\video_comparison_3_45PM\raw_data\extracted_data\francis_10min_ch_1.csv'
# file2 = r'C:\Users\qchen\PARC\Fibridge-PARC - Drive Easy\AustraliaDeploy\Francis\video_comparison_3_45PM\raw_data\extracted_data\francis_10min_ch_2.csv'
# df = pd.read_csv(file1)
# df2 = pd.read_csv(file2)

In [ ]:
# exam the loaded data
# set index of sensor to look at
import matplotlib.dates as mdates


k = 3
fig = plt.figure(1)
plt.clf()
ax = fig.add_subplot(111)

# if there is a built-in timestamp
ax.plot(df.iloc[:40000, 0].astype('datetime64'), np.asarray(df.iloc[:40000, k])-df.iloc[0,k], label='ch_1')
ax.plot(df2.iloc[:40000, 0].astype('datetime64'), np.asarray(df2.iloc[:40000, k])-df2.iloc[0,k], label='ch_2')

# ax.xaxis.set_major_formatter(mdates.DateFormatter('%d %H-%M-%s-%f'))
# fig.autofmt_xdate()

# ax.plot(np.asarray(df.iloc[:40000, k])-df.iloc[0,k], label='ch_1')
# ax.plot(np.asarray(df2.iloc[:40000, k])-df2.iloc[0,k], label='ch_2')
plt.legend()


In [ ]:
event_vis = np.zeros([11000, 25])

# for j in range(25):
#     event_vis_temp = event_detection(df.iloc[:, j], threshold=THRESHOLD)
#     event_vis[:len(event_vis_temp), j] = np.asarray(event_vis_temp)
#     print('Processing sensor #{}'.format(j))

for j in range(25):
#     trace_temp = de_trend(df.iloc[:, j]) # detrend or not for initial detection
    trace_temp = df.iloc[:, j+1]
    event_vis_temp = event_detection(trace_temp, threshold=THRESHOLD)
    event_vis[:len(event_vis_temp), j] = np.asarray(event_vis_temp)
    print('Processing sensor #{}'.format(j))

In [ ]:
print(len(event_vis_temp))
event_vis = event_vis[:len(event_vis_temp)]
plt.figure(2)
plt.clf()
plt.title('Event detection map')
for j in range(25):

    plt.plot(event_vis[:, j] + 1*j)

In [ ]:
lane_sensor = [0,1,2,3,4,5,6,7,8,9,10]  # for lane 5 on M80
# lane_sensor= [11,12,13,14,15,16,17,18,19,20]

combined_event_vis = np.sum(event_vis[:,lane_sensor], axis=1)
combined_event_vis = combined_event_vis[:len(event_vis_temp)]

plt.figure(3)
plt.clf()
timeseg = np.linspace(0, len(event_vis_temp)*BIN_SIZE, len(event_vis_temp))
plt.plot(timeseg, combined_event_vis+1)
plt.plot((np.asarray(df.iloc[:, lane_sensor[4]])-df.iloc[0,lane_sensor[4]])*1000)



plt.figure(4)
plt.clf()
plt.plot(timeseg/SAMPLING_RATE, combined_event_vis, label='event detection by individual sensors')
# plt.plot((timeseg/SAMPLING_RATE)[:len(event_vis_agr)], np.asarray(event_vis_agr)-1.2, label='event detection by aggregated signal')
# plt.legend()
peaks = signal.find_peaks(combined_event_vis, width=1)

plt.plot(peaks[0]*BIN_SIZE/SAMPLING_RATE, peaks[1]['width_heights'], '*r')
plt.ylabel('# of responding sensors')
plt.xlabel('Time (s)')
print(df.iloc[0,0])
print('number of peaks{}.'.format(len(peaks[0])))

In [ ]:
event_list = []
ap_bin = 1

peaks = signal.find_peaks(combined_event_vis, width=1)
lane_sensor = [0,1,2,3,4,5,6,7,8,9,10]
# lane_sensor= [11,12,13,14,15,16,17,18,19,20]
for i in range(len(peaks[0])):
    event_temp = Event()
    event_temp.sensor = lane_sensor
    event_temp.start = int(peaks[1]['left_bases'][i]*BIN_SIZE)
    event_temp.end = int(peaks[1]['right_bases'][i]*BIN_SIZE)
    if len(event_list)>0:
        if event_temp.start < event_list[-1].end-BIN_SIZE:
            print('overlength event #{}'.format(i))
            continue
    event_temp.data['leading'] = df.iloc[event_temp.start:event_temp.end+BIN_SIZE*ap_bin, 1:26]
    event_temp.data['leading'] = event_temp.data['leading'].reset_index(drop=True)
    event_temp.data['trailing'] = df2.iloc[event_temp.start:event_temp.end+BIN_SIZE*ap_bin, 1:26]
    event_temp.data['trailing'] = event_temp.data['trailing'].reset_index(drop=True)
    event_temp.data['timestamp'] = df.iloc[event_temp.start:event_temp.end+BIN_SIZE*ap_bin, 0]
    event_temp.data['timestamp'] = event_temp.data['timestamp'].reset_index(drop=True)
    event_temp.timestamp = df.iloc[event_temp.start, 0]
    event_temp.calculate_speed(event_vis, sn=25, offset=0)
    if event_temp.speed == 0:
        print('speed error at {}'.format(i))
        continue
    if event_temp.speed < 0:
        print('vehicle at opposite lane')
        continue
    event_temp.index = combined_event_vis[peaks[0][i]]
    event_list.append(event_temp)
print('Total event detected: {}'.format(len(event_list)))

In [ ]:
# save png to folder
# for j in range(len(event_list)):
# #     print('Total:{}. Plotting #{}'.format(len(event_list), j))
#     plt.figure(5)
#     plt.clf()
#     for i in lane_sensor:
#         plt.plot(np.asarray(event_list[j].data['leading'].iloc[:,i]) - event_list[j].data['leading'].iloc[0,i] + 0.001 * i)
#         plt.title('event #{}. Timestamp: {}. Estimated speed {:2.2f} KPH'.format(j, event_list[j].timestamp, event_list[j].speed))
#     plt.savefig('Event#{}_ch3_first12.png'.format(j))
#     plt.clf()
#     for i in lane_sensor:
#         plt.plot(np.asarray(event_list[j].data['trailing'].iloc[:,i]) - event_list[j].data['trailing'].iloc[0,i] + 0.001 * i)
#         plt.title('event #{}. Timestamp: {}. Estimated speed {:2.2f} KPH'.format(j, event_list[j].timestamp, event_list[j].speed))
#     plt.savefig('Event#{}_ch4_first12.png'.format(j))

In [ ]:
# View extracted event trace
# print(event_temp.data['timestamp'])
j = 1 # event index
print('Total:{}. Plotting #{}'.format(len(event_list), j))
print('active sensor:{}'.format(len(event_list[j].sensor_active)))
plt.figure(6)
plt.clf()
for i in event_list[j].sensor:
#     plt.plot(event_list[j].data['timestamp'].astype('datetime64'), np.asarray(event_list[j].data['leading'].iloc[:,i]) - event_list[j].data['leading'].iloc[0,i] + 0.001 * i)
    plt.plot(np.asarray(event_list[j].data['leading'].iloc[:,i]) - event_list[j].data['leading'].iloc[0,i] + 0.001 * i)
    plt.title('event #{}. Timestamp: {}. Estimated speed {:2.2f} KPH'.format(j, event_list[j].timestamp, event_list[j].speed))

plt.figure(7)
plt.clf()
for i in event_list[j].sensor:
#     plt.plot(event_list[j].data['timestamp'].astype('datetime64'), np.asarray(event_list[j].data['trailing'].iloc[:,i]) - event_list[j].data['trailing'].iloc[0,i] + 0.001 * i)
    plt.plot(np.asarray(event_list[j].data['trailing'].iloc[:,i]) - event_list[j].data['trailing'].iloc[0,i] + 0.001 * i)
    plt.title('event #{}. Timestamp: {}. Estimated speed {:2.2f} KPH'.format(j, event_list[j].timestamp, event_list[j].speed))

print(event_list[j].sensor)
event_list[j].speed_alg3(0)
print(event_list[j].speed)
plt.figure(11)
plt.clf()

trace_temp = np.sum(event_list[j].data['leading'].iloc[:,1:2], axis=1)
trace_temp = np.abs(trace_temp-trace_temp[0])
peaks_temp=signal.find_peaks(trace_temp, height=0.01)
plt.plot(trace_temp)
plt.plot(peaks_temp[0], 0.002*np.ones(len(peaks_temp[0])),'*b')
peaks_temp[0]
trace_temp = np.sum(event_list[j].data['trailing'].iloc[:,1:2], axis=1)
trace_temp = np.abs(trace_temp-trace_temp[0])
peaks_temp=signal.find_peaks(trace_temp, height=0.01)
plt.plot(trace_temp)
plt.plot(peaks_temp[0], 0.002*np.ones(len(peaks_temp[0])),'*r')

In [ ]:
ll = 19
speed_valid = signal.correlate(event_list[j].data['leading'].iloc[:,ll]-event_list[j].data['leading'].iloc[0,ll], event_list[j].data['trailing'].iloc[:,ll]-event_list[j].data['trailing'].iloc[0,ll])
speed = -1*2.5/(speed_valid.argmax()-len(event_list[j].data['leading']))*SAMPLING_RATE*3.6
print(speed)
plt.figure()
plt.plot(speed_valid)

In [ ]:
print('Plotting sensor #{} of event#{}'.format(k, j))
k= 1 # sensor number
plt.figure(8)
plt.clf()
plt.plot(np.asarray(event_list[j].data['leading'].iloc[:,k]) - event_list[j].data['leading'].iloc[0,k])
plt.plot(np.asarray(event_list[j].data['trailing'].iloc[:,k]) - event_list[j].data['trailing'].iloc[0,k] + 0.002)

In [ ]:
plt.figure(9)
event_speed = [event_list[k].speed for k in range(len(event_list))]
event_time = [event_list[k].data['timestamp'][0] for k in range(len(event_list))]
df_time=pd.DataFrame(event_time, columns=['timestamp'])
df_time['timestamp']=df_time['timestamp'].astype("datetime64")


plt.stem(df_time['timestamp'], event_speed, use_line_collection=True)
plt.xticks(rotation=0)
plt.ylabel('Speed (KPH)')

plt.figure(10)
plt.clf()
plt.stem(event_speed, use_line_collection=True)

In [ ]:
plt.figure()
event_confid = [event_list[k].index for k in range(len(event_list))]
plt.stem(df_time, np.ones(len(event_list)), use_line_collection=True)
fig = plt.gca()
plt.xticks(rotation=25)
plt.ylabel('Vehicle detected')